### OTFS Modulation Scheme

The Orthogonal Frequency Time Space (OTFS) is a modulation scheme used to improve and tackle the problems which traditional OFDM systems cannot. This is done by shifting the analysis and synthesis operations to the delay-doppler domain.


In [ ]:
import commpy
import numpy as np
import matplotlib.pyplot as plt

### OTFS Block Diagram

The entire process of OTFS can be summarized by the following block diagram :

<img src="Figures/OTFS_image.PNG" width="240*4" height="240*4" align="center"/>

<i> Orthogonal Time Frequency Space (OTFS) Modulation
and Applications,Tutorial at SPCOM 2020, IISc, Bangalore, July, 2020
Yi Hong, Emanuele Viterbo, Raviteja Patchava </i>

As shown in the above block diagram we can breakdown the OTFS scheme into the following steps :

1. Generation of symbols (QPSK,QAM etc.,) these are automatically assumed to be in the delay-doppler domain.
1. Conversion of these symbols into the time-frequency domain : ISFFT
1. Time-frequency domain to time domain i.e. transmission signal s(t) : Heisenberg Transform
1. Transmission over channel : Channel modelled in delay-doppler domain
1. Received signal converted into time-frequency domain using Wigner transform done through matched filtering.
1. Time frequency domain to delay doppler domain using SFFT.

### Building the system in the discrete domain

![Matrix](Figures/OTFS_matrix.PNG)

In [ ]:
def gen_bits(size):
    return np.random.randint(2,size=(size,))

In [ ]:
M = 2048 # Number of points in the delay domain
N=  128  # Number of points in the doppler domain

total_symbols = M*N # Total Number of symbols that are required to be genrated for the given M and N
no_bits = 2*total_symbols # 4-QAM modulation is chosen i.e. 2 bits for every symbol

bits = gen_bits(no_bits) # Generating the information bits stream
QAM = commpy.QAMModem(4) # Creating the 4-QAM object
sym = QAM.modulate(bits) # Modulating the input bit stream into symbols

In [ ]:
sym_delay_dop = np.reshape(sym,(M,N))               # Constructing the symbol matrix
sym_delay_time = np.zeros((M,N), dtype = 'complex') # Initialising the delay time matrix 

for i in range(M) :                                 # Creating the delay timematrix by taking row wise fft            
    sym_delay_time[i,:] = np.fft.fft(sym_delay_dop[i,:])                    

In [ ]:
## Parallel to Serial Convertor
CP = 16  # Number of cyclic prefix added as a gaurd interval to prevent ISI 
x = np.zeros((M*N,), dtype = 'complex') # Creating the transmission array after P/S
x_CP = np.zeros((M*N + CP,), dtype = 'complex') # Adding CP 
for i in range(N):
    x[M*i:M*(i+1)] = sym_delay_time[:,i]
x_CP[:-CP] = x[::-1] # Reversing the x since we want the first symbol transmitted to be N=0 and not N=128
x_CP[-CP:] = x_CP[:CP] # Adding the cyclic prefix 

### Creating the Channel 

We now need to model the channel in the delay doppler domain. The following have to be decided to construct the channel : 

1. Number of taps
1. Channel model